In [103]:
import json
import pandas as pd
import ast
import datetime
from glob import glob
import unidecode

In [104]:
files = sorted(glob("results/merged_*.csv"))

def get_date(s):
    return s.split("merged_")[1].split(".csv")[0]

infile1 = files[0]
infile2 = files[1]
col_to_export = ["venezuelans","allpopulation"] #,'audience_adolecent', 'audience_young_adult','audience_adult', 'audience_middle_age','audience_elder',
                                                #'audience_locals_adolecent', 'audience_locals_young_adult','audience_locals_adult', 'audience_locals_middle_age','audience_locals_elder']

df1 = pd.read_csv(infile1)
df2 = pd.read_csv(infile2)
merged = pd.merge(df1[["Location","LocationHierarchy","Frequency", *col_to_export]], df2[["Location","LocationHierarchy","Frequency", *col_to_export]], 
                  on=["Location","LocationHierarchy","Frequency"], suffixes=(get_date(files[0]), (get_date(files[1]))), how="outer")

for i in range(2, len(files)):
    tmp = pd.read_csv(files[i])
    merged = pd.merge(merged, 
          tmp[["Location","LocationHierarchy","Frequency", *col_to_export]], 
          on=["Location","LocationHierarchy","Frequency"], suffixes=("", get_date(files[i])), how="outer")
    
for col in col_to_export:
    merged[col + get_date(files[2])] = merged[col]
    del merged[col]
    
    
merged = merged[merged["Frequency"] == "Monthly"]
   

In [105]:
vencols = [c for c in merged.keys() if c.startswith("venezuelans")]
merged["venezuelans_hist_min"] = merged[vencols].min(axis=1)
merged["venezuelans_hist_max"] = merged[vencols].max(axis=1)
merged.to_csv("till_2019_06_19.csv")

In [106]:
# Creates final merged file
dfacc = "till_2019_06_19.csv"
latest = "./results/merged_19_06_18.csv"

pendular = "./results/pendular_19_06_18.csv"
married = "./results/married_19_06_18.csv"
immigrant = "./results/immigrants_19_06_18.csv"

dfacc = pd.read_csv(dfacc)
latest = pd.read_csv(latest)

pendular = pd.read_csv(pendular)
del pendular["audience_Expats (Venezuela)"]
married = pd.read_csv(married)
immigrant = pd.read_csv(immigrant)

merged_all = pd.merge(dfacc, latest, how="outer", on=["Location","LocationHierarchy","Frequency"], suffixes=(get_date(files[0]), (get_date(files[1]))))
merged_all = pd.merge(merged_all, pendular, how="outer", on=["Location","LocationHierarchy","Frequency"])
merged_all = pd.merge(merged_all, married, how="outer", on=["Location","LocationHierarchy","Frequency"])
merged_all = pd.merge(merged_all, immigrant, how="outer", on=["Location", "Frequency"])

del merged_all["Unnamed: 0"]


merged_all = merged_all[merged_all["Frequency"] == "Monthly"]

#merged_all["Location"] = merged_all["Location"].apply(lambda x : unidecode.unidecode(x))
merged_all.fillna(0.0).to_csv("all_2019_06_19+.csv", index=False)



In [141]:
countries = merged_all[(merged_all["LocationHierarchy"] == "Country") & (merged_all["Frequency"] == "Monthly")].copy()

unhcr_countries = ['Brazil', 'Uruguay', 'Paraguay', 'Argentina', 'Chile', 'Bolivia', 'Peru', 'Ecuador', 'Colombia',
                  'Panama', 'Costa Rica', 'Dominican Republic', 'Aruba', 'Curacao', 'Trinidad and Tobago', 'Guyana', 'Mexico']

countries = countries[countries["Location"].apply(lambda x: x in unhcr_countries)]

#countries[["allpopulation19_03_01", "allpopulation19_03_20"]].sum()
#countries[["venezuelans19_03_01", "venezuelans19_03_20"]].sum()
countries["20/01"] = countries["venezuelans19_03_20"] /countries["venezuelans19_03_01"]
countries[["Location", "venezuelans19_03_01", "venezuelans19_03_20", "20/01"]]

valid = countries[countries["venezuelans19_03_20"] > 1000]
valid[["Location", "venezuelans19_03_14", "venezuelans19_03_20", "venezuelans19_03_17", "20/01"]].sum()

#valid[["venezuelans19_03_01", "venezuelans19_03_20", "20/01"]].corr("spearman")

Location               ColombiaBrazilPeruPanamaChileEcuadorArgentinaC...
venezuelans19_03_14                                           3.2548e+06
venezuelans19_03_20                                           2.3653e+06
venezuelans19_03_17                                           2.4777e+06
20/01                                                            13.2175
dtype: object

In [109]:
countries = merged_all[(merged_all["Frequency"] == "Monthly")].copy()

#countries[["allpopulation19_03_01", "allpopulation19_03_20"]].sum()
#countries[["venezuelans19_03_01", "venezuelans19_03_20"]].sum()
countries["20/01"] = countries["venezuelans19_03_20"] /countries["venezuelans19_03_01"]
countries[["Location", "venezuelans19_03_01", "venezuelans19_03_20", "20/01"]]

valid = countries[countries["venezuelans19_03_20"] > 1000]
valid[["Location", "venezuelans19_03_01", "venezuelans19_03_20", "20/01"]]["20/01"].mean()

valid[["venezuelans19_03_01", "venezuelans19_03_20", "20/01"]].corr()

,venezuelans19_03_01,venezuelans19_03_20,20/01
venezuelans19_03_01,1.000000,0.997684,-0.141861
venezuelans19_03_20,0.997684,1.000000,-0.121781
20/01,-0.141861,-0.121781,1.000000
